In [1]:
import sys
sys.path.append('../')
from rocket_rag.utils import *
from rocket_rag.node_indexing import *
from rocket_rag.vector_store import *

c:\Users\s381731\AppData\Local\miniconda3\envs\agents\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load = '40kg'
if_files_dict = parse_files(main_directory=INFERENCE_DIR)
if_ts_files = if_files_dict[load]

In [5]:
# np.random.seed(42)

rand_idx = np.random.randint(0, len(if_ts_files))
if_ts_filename = if_ts_files[rand_idx]
print(f'Random inference sample: {if_ts_filename}')
if_rocket_feature = fit_transform([if_ts_filename],
                                    field='current',
                                    smooth=True,
                                    smooth_ws=15,
                                    tolist=False,
                                    verbo=False)
print(f'ROCKET features shape: {if_rocket_feature.shape}')

Random inference sample: ../data/inference/40kg\lackLubrication2\lackLubrication2_40_7_4.csv
ROCKET features shape: (1, 20000)


In [6]:
node_indexer = NodeIndexer()
nodes = node_indexer.load_node_indexing(f'../store/nodes_{load}.pkl')

2024-03-17 14:21:01.258 | DEBUG    | rocket_rag.node_indexing:load_node_indexing:98 - Loading all nodes...
2024-03-17 14:21:01.975 | INFO     | rocket_rag.node_indexing:load_node_indexing:102 - All nodes are loaded.


In [7]:
vector_store = VectorStore()
vector_store.add(nodes)

In [8]:
s, ids = vector_store.knn_query(if_rocket_feature, k=5)
print(s)
print(ids)

['2.4026516730675422', '2.6995707092654024', '2.8018250691051385', '2.9966474927072353', '3.3901454900633676']
['lackLubrication2_40_8_5', 'lackLubrication2_40_3_5', 'lackLubrication2_40_9_3', 'lackLubrication2_40_3_2', 'lackLubrication2_40_2_2']


### Get the fault diagnosis reuslt

In [9]:
from rocket_rag.prompts import fault_diagnosis_prompt

In [10]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

history = [
    {"role": "system", "content": fault_diagnosis_prompt.sys_prompt},
    {"role": "user", "content": fault_diagnosis_prompt.user_prompt.format(res=str(ids), score=s)},
]

completions = client.chat.completions.create(
    model="local-model", # this field is currently unused
    messages=history,
    temperature=0.1,
    stream=True,
)

new_message = {"role": "assistant", "content": ""}

for chunk in completions:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        new_message["content"] += chunk.choices[0].delta.content

history.append(new_message)
fault_diagnosis_res = new_message['content']

Retrieval results: ['lackLubrication2_40_8_5', 'lackLubrication2_40_3_5', 'lackLubrication2_40_9_3', 'lackLubrication2_40_3_2', 'lackLubrication2_40_2_2']
Diagonsis results:
Refined fault type1: Fault in lack of lubrication
Inference evidence: ['lackLubrication2_40_8_5' with similarity score 3.39, 'lackLubrication2_40_3_5' with similarity score 2.69, 'lackLubrication2_40_9_3' with similarity score 2.8, 'lackLubrication2_40_3_2' with similarity score 2.99, 'lackLubrication2_40_2_2' with similarity score 2.4]
Description of the Fault: The actuator is experiencing a lack of lubrication, which can lead to increased friction and potential wear and tear, impacting performance and longevity. This condition may worsen over time if not addressed promptly. It's recommended to schedule maintenance and ensure proper lubrication for the actuator to maintain optimal operation.

### Use multi-query generation for query parsing 

In [11]:
from rocket_rag.prompts import multi_queries_gen_prompt

In [12]:
mq_messages = [
    {"role": "system", "content": multi_queries_gen_prompt.sys_prompt},
    {"role": "user", "content": multi_queries_gen_prompt.user_prompt.format(res=fault_diagnosis_res, num=5)},
]
            
completions = client.chat.completions.create(
    model="local-model", # this field is currently unused
    messages=mq_messages,
    temperature=0.1,
    stream=True,
)

new_message = {"role": "assistant", "content": ""}

for chunk in completions:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        new_message["content"] += chunk.choices[0].delta.content

history.append(new_message)
multi_queries_gen = new_message['content']

1. "Linear actuator lack of lubrication maintenance: methods and best practices"
2. "How often should I lubricate my linear actuator? Frequency and techniques"
3. "Troubleshooting linear actuator: identifying and resolving lack of lubrication issues"
4. "Linear actuator lubrication guide: types, tools, and procedures"
5. "Maintaining optimal performance for linear actuators: addressing lack of lubrication concerns"

In [13]:
import re

def formalize_query(query: str):
    """Preprocess the query for the vector store query
    
    Remove some symbols including '-' and indexing numbers or patterns like 1. 2. 3. ...
    """
    query = query.strip().replace('"', '').replace('. ', '')
    pattern = re.compile(r'[-0-9]+|\d+\. ')
    result = pattern.sub('', query)
    return result.strip()

In [14]:
generated_queries = [formalize_query(query) for query in multi_queries_gen.split('\n')]
generated_queries

['Linear actuator lack of lubrication maintenance: methods and best practices',
 'How often should I lubricate my linear actuator? Frequency and techniques',
 'Troubleshooting linear actuator: identifying and resolving lack of lubrication issues',
 'Linear actuator lubrication guide: types, tools, and procedures',
 'Maintaining optimal performance for linear actuators: addressing lack of lubrication concerns']

### Use external tools for fault diagnosis maintenance support
Here use Google chrome web browser for a proof-of-concept validation

In [ ]:
import json
from googleapiclient.discovery import build

CONFIG_FILE = "../../config/config.json"
with open(CONFIG_FILE) as f:
    config = json.load(f)
    GOOGLE_API_KEY = config["google_api_key"]
    GOOGLE_CSE_ID = config["google_cse_id"]

In [18]:
def call_google(query: str, **kwargs):
    """ Call the google chrome for searching online """
    
    service = build(serviceName="customsearch", 
                    version="v1", 
                    developerKey=GOOGLE_API_KEY,
                    static_discovery=False)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, **kwargs).execute()
    res_items = res["items"]
    res_snippets = [r['snippet'] for r in res_items]
    return str(res_snippets)

# A quick validation for the call_google
# print(call_google(query=generated_queries[0]))

In [20]:
available_functions = {"call_google": call_google}

In [ ]:
from rocket_rag.prompts import react_prompt
from colorama import Fore, Back, Style

# regular expression regex patterns
action_re = re.compile('^Action: (\w+): (.*)$')
answer_re = re.compile("Answer: ")
answers = []

chrome_messages = [
    {"role": "system", "content": react_prompt.sys_prompt},
    {"role": "user", "content": react_prompt.user_prompt.format(query=generated_queries[0])},
]

while True:
    response = client.chat.completions.create(
        model="local-model",
        messages=chrome_messages,
    )
    
    # Get the response from the GPT and add it as a part of memory
    response_msg = response.choices[0].message.content
    history.append({"role": "assistant", "content": response_msg})
    
    # If the respionse contains the keyword "Answer: ", then return
    if answer_re.search(response_msg):
        print(Fore.YELLOW + response.choices[0].message.content)
        print(Style.RESET_ALL)
        answers.append(response_msg)
        break
    
    # Print the thinking process
    print(Fore.GREEN + response_msg)
    print(Style.RESET_ALL)

    # Take actions
    actions = [action_re.match(a) for a in response_msg.split("\n") if action_re.match(a)]
    if actions:
        action, action_input = actions[0].groups()
        try:
            print(Fore.CYAN + f" -- running {action} {action_input}")
            print(Style.RESET_ALL)
            # Apply available tools for the function execution
            obervation = available_functions[action](action_input) 
            print(Fore.BLUE + f"Observation: {obervation}")
            print(Style.RESET_ALL)
            history.append({"role": "user", "content": "Observation: " + obervation})
        except:
            raise NotImplementedError